In [202]:
import pandas as pd
import os

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

import category_encoders as ce

pd.options.display.max_columns = 1999

In [192]:
def custom_metric(actuals, forecast, avg_volume):
    """
    This function aims to compute the Custom Accuracy Metric
    for the Novartis Datathon, 3rd edition.

    Given the actuals followed by the forecast and the avg_volume
    of the brand, it will compute the metric score.

    Keyword parameters:
        actuals (float vector): Real value of Y
        forecast (float vector): Volume forecast
        avg_volume (float): Average monthly volume of the 12 months
                            prior to the generic entry.

    Returns:
        custom_metric: Uncertainty Metric score (%)
    """

    # Compute the first part of the equation
    # (custom MAPE with Average volume)
    custom_mape = sum(abs(actuals - forecast)) / (24 * avg_volume)

    # Compute the second part of the equation
    # (custom 6-first-months MAPE with Average volume)
    six_month_mape = \
        abs(sum(actuals[:6]) - sum(forecast[:6])) / (6 * avg_volume)

    # Compute the third part of the equation
    # (custom 6-months MAPE with Average volume)
    twelve_month_mape = \
        abs(sum(actuals[6:12]) - sum(forecast[6:12])) / (6 * avg_volume)

    # Compute the fourth part of the equation
    # (custom 12-months MAPE with Average volume)
    last_month_mape = \
        abs(sum(actuals[12:]) - sum(forecast[12:])) / (12 * avg_volume)

    # Compute the custom metric
    custom_metric = 0.5 * custom_mape + 0.3 * six_month_mape + \
        0.1 * (twelve_month_mape + last_month_mape)

    return custom_metric * 100

In [176]:
generics_count = pd.read_csv('./data/gx_num_generics.csv')
package = pd.read_csv('./data/gx_package.csv')
gx_volume = pd.read_csv('./data/gx_volume.csv')

In [177]:
data_merged = pd.read_csv('/Users/jakob/Desktop/dt_merged_w.csv')

In [178]:
data_merged.drop(['i', 'max_mon'], axis=1, inplace=True)

In [179]:
data_merged

,brand,therap,country,A,B,C,D,package,num_generics,test,-137,-136,-135,-134,-133,-132,-131,-130,-129,-128,-127,-126,-125,-124,-123,-122,-121,-120,-119,-118,-117,-116,-115,-114,-113,-112,-111,-110,-109,-108,-107,-106,-105,-104,-103,-102,-101,-100,-99,-98,-97,-96,-95,-94,-93,-92,-91,-90,-89,-88,-87,-86,-85,-84,-83,-82,-81,-80,-79,-78,-77,-76,-75,-74,-73,-72,-71,-70,-69,-68,-67,-66,-65,-64,-63,-62,-61,-60,-59,-58,-57,-56,-55,-54,-53,-52,-51,-50,-49,-48,-47,-46,-45,-44,-43,-42,-41,-40,-39,-38,-37,-36,-35,-34,-33,-32,-31,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,brand_1,Nervous_system,country_12,NaN,4.086695,NaN,95.913305,PILL,25,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257179,0.240887,0.245431,0.265500,0.271535,0.269382,0.292642,0.280513,0.349966,0.321104,0.354295,0.346548,0.362649,0.331070,0.332588,0.379997,0.365481,0.355142,0.398845,0.394533,0.402781,0.441539,0.413772,0.421545,0.454930,0.375371,0.447824,0.457890,0.419393,0.465527,0.453720,0.413584,0.460827,0.473180,0.458033,0.489217,0.515069,0.425046,0.488183,0.487834,0.478158,0.525930,0.484787,0.482062,0.582125,0.526825,0.538536,0.578013,0.611379,0.543496,0.587805,0.597072,0.600953,0.606445,0.596334,0.577363,0.690244,0.642154,0.674841,0.701900,0.699218,0.661779,0.691958,0.680635,0.618279,0.792606,0.713479,0.710594,0.781600,0.736703,0.801588,0.815690,0.755903,0.733564,0.731491,0.835934,0.788389,0.790931,0.849151,0.778444,0.797366,0.872732,0.862792,0.827001,0.939524,0.824263,0.853856,0.935625,0.843503,0.880230,0.947375,0.850931,0.926276,0.938256,0.930648,0.907464,1.000050,0.862519,0.962146,1.008478,0.923436,1.001614,1,0.878691,0.962668,0.891506,0.881312,0.904497,0.925172,0.811468,0.867441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,brand_1,Nervous_system,country_15,NaN,11.585173,NaN,88.414827,PILL,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200011,0.216883,0.211548,0.222075,0.226352,0.222892,0.229677,0.207884,0.230063,0.226941,0.254367,0.246689,0.259456,0.257793,0.247549,0.281729,0.283804,0.289429,0.285395,0.274977,0.283306,0.301125,0.304233,0.321527,0.340147,0.315829,0.342348,0.366696,0.346089,0.395349,0.357416,0.343548,0.403344,0.398547,0.388712,0.429662,0.448091,0.415969,0.441439,0.450975,0.439731,0.490278,0.441230,0.420488,0.499743,0.478556,0.468876,0.510814,0.530022,0.515400,0.549341,0.541493,0.569853,0.611951,0.533752,0.516822,0.608966,0.562472,0.594641,0.615041,0.595201,0.629248,0.642849,0.621760,0.666585,0.674910,0.603535,0.616325,0.669552,0.619674,0.712108,0.643620,0.693117,0.708722,0.662332,0.737301,0.719263,0.724828,0.745017,0.676221,0.743307,0.765590,0.787500,0.725760,0.822745,0.792577,0.779209,0.844497,0.807113,0.794157,0.852204,0.773442,0.838274,0.847267,0.863149,0.852256,0.934323,0.850743,0.929645,0.944801,0.870354,1,0.743884,0.409105,0.444838,0.390535,0.386130,0.372340,0.379942,0.343170,0.362080,0.363910,0.339834,0.390581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,brand_1,Nervous_system,country_16,NaN,10.043531,1.563480,88.392989,PILL,1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.521929,0.526968,0.676595,0.547675,0.546253,0.693114,0.541719,0.549259,0.713876,0.573855,0.588542,0.717103,0.582936,0.583797,0.743144,0.624525,0.608420,0.769735,0.620508,0.635487,0.828613,0.675365,0.697784,0.868524,0.703350,0.718334,0.932559,0.766190,0.794312,0.927201,0.817035,0.833912,1.064670,0.847064,0.860956,1.100817,0.858268,0.874259,1.118506,0.893225,0.896336,1.112634,0.872829,0.852014,1.090750,0.848933,0.883467,1.080291,0.835198,0.854452,1.058370,0.885785,0.865769,1.087348,0.886521,0.867290,1.122895,0.891389,0.924267,1.093231,0.834

In [180]:
data_merged = data_merged[data_merged['test'] == False]

In [181]:
data_merged[['A', 'B', 'C', 'D']] = data_merged[['A', 'B', 'C', 'D']].fillna(0)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [182]:
X = data_merged.iloc[:, :9]
y = data_merged.iloc[:, -24]

In [183]:
encoder = ce.TargetEncoder(cols=['brand'])
encoder.fit(X, y)
X = encoder.transform(X)

In [184]:
X = pd.get_dummies(X)

In [185]:
X

,brand,A,B,C,D,num_generics,therap_Anti_infectives,therap_Antineoplastic_and_immunology,therap_Cardiovascular_Metabolic,therap_Dermatology,therap_Endocrinology_and_Metabolic_Disease,therap_Haematology,therap_Muscoskeletal_Rheumatology_and_Osteology,therap_Nervous_system,therap_Obstetrics_Gynaecology,therap_Other,therap_Parasitology,therap_Respiratory_and_Immuno_inflammatory,therap_Sensory_organs,therap_Systemic_Hormones,country_country_1,country_country_10,country_country_11,country_country_12,country_country_14,country_country_15,country_country_16,country_country_2,country_country_3,country_country_4,country_country_5,country_country_7,country_country_8,country_country_9,package_CREAM,package_EYE_DROP,package_INJECTION,package_OTHER,package_PATCH,package_PILL
6,0.806737,0.0,36.170513,0.000000,63.829487,17,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
7,0.855763,0.0,1.015697,0.000000,98.984303,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,0.855763,0.0,0.008023,0.000000,99.991977,9,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,0.855763,0.0,3.722347,0.000000,96.277653,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
10,0.855763,0.0,4.681257,1.406766,93.911977,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,0.782186,0.0,1.745928,0.000000,98.254072,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1073,0.782186,0.0,3.230206,0.000000,96.769794,15,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
1074,0.782186,0.0,17.941044,82.058956,0.000000,20,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1076,0.806737,0.0,0.169352,0.000000,99.830648,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [186]:
y

6       0.986533
7       0.866219
8       0.923390
9       0.889393
10      0.753812
          ...   
1072    0.282406
1073    0.971551
1074    0.875032
1076    0.927280
1077    0.870178
Name: 0, Length: 887, dtype: float64

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [197]:
regressor = DecisionTreeRegressor(random_state=0, criterion='mae')
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mae', random_state=0)

In [203]:
y_pred = regressor.predict(X_test)
mean_absolute_error(y_test, y_pred)

0.15723506237419027